# 1. 导入模块和库

In [3]:
import time
import csv
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix, classification_report

E:\Anaconda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. 导入数据+预处理+降维

In [4]:
data = pd.read_csv(r"F:\Code_Data\2_Rocks_Spectrum_Reflectance_essay_30_origin_modify_2.csv",encoding='utf-8') #encoding='GBK',防止中文乱码
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]
X_SG = savgol_filter(X, 5, 2)
X_SG_mms = MinMaxScaler().fit_transform(X_SG)
Label = LabelEncoder().fit_transform(y)
data.iloc[:,1:-1] = X_SG_mms
data.iloc[:, -1] = Label
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]
X.shape, y.shape

X_dr = PCA(29).fit_transform(X)

# 3. 遗传算法调参

In [10]:
# （1）构建目标函数及定义参数空间
def optuna_objective(trial):

    #定义参数空间
    n_estimators = trial.suggest_int("n_estimators",1,150,1) #整数型，(参数名称，下界，上界，步长)
    max_depth = trial.suggest_int("max_depth",30,50,1)
    max_features = trial.suggest_int("max_features",5,30,1)
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,51,1)
    min_samples_split = trial.suggest_int("min_samples_split",2,22,1)

    #定义评估器
    #需要优化的参数由上述参数空间决定
    #不需要优化的参数则直接填写具体值
    rfc = RFC(n_estimators = n_estimators
              ,max_depth = max_depth
              ,max_features = max_features
              ,min_samples_leaf = min_samples_leaf
              ,min_samples_split = min_samples_split
              ,random_state=0
              ,verbose=False
              ,n_jobs=-1
             )

    #交叉验证过程，输出准确率(accuracy)
    cv = KFold(n_splits=5,shuffle=True,random_state=0)
    score = cross_val_score(rfc,X_dr,y
                            ,scoring='accuracy'
                            ,cv=cv #交叉验证模式
                            ,verbose=False #是否打印进程
                            ,n_jobs=-1 #线程数
                            ,error_score='raise' #如果在某个折叠中出现错误，会引发一个异常
                           )
    #最终输出平均准确率
    return np.mean(score)


#（2）定义优化目标函数的具体流程
def optimizer_optuna(n_trials, algo):

    # 检查是否已经存在study，存在则删除，不存在则略过    
    try:
        storage = "sqlite:///random_GAO"  # Specify the storage URL here.
        study_name = "random_GAO"
       
        # 获取 study 的 ID
        storage = optuna.storages.get_storage(storage)
        study_id = storage.get_study_id_from_name(study_name)
       
        # 删除已存在的 study
        storage.delete_study(study_id)
        print(f"Deleted existing study: {study_name}")
    except KeyError as e:
        print(f"Study '{study_name}' does not exist.")

    #定义使用遗传算法搜索
    sampler = optuna.samplers.NSGAIISampler(population_size=50  # int，种群大小，即每代中包含的个体数量。默认 50
                                           ,mutation_prob=1.0 # float，变异概率，指定了在交叉之后每个个体发生变异的概率。默认 1.0。
                                           ,crossover_prob=1.0  # float，交叉概率，指定了在每次交叉操作中，一个个体与另一个个体产生子代的概率。默认 1.0
                                           )

    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = sampler #要使用的具体算法
                                ,direction="maximize" #优化的方向，可以填写minimize或maximize
                                ,storage="sqlite:///random_GAO" # Specify the storage URL here.
                                ,study_name="random_GAO"
                               )
    study.optimize(optuna_objective #目标函数
                   ,n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   ,show_progress_bar=True #要不要展示进度条呀？
                  )

    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")

    return study.best_trial.params, study.best_trial.values

#（3）执行优化操作
def optimized_optuna_search_and_report(n_trials, algo):
    start_time = time.time()
    
    best_params, best_score = optimizer_optuna(n_trials, algo)

    # 打印最佳参数和分数
    print("\n","\n","best params: ", best_params,
          "\n","\n","best score: ", best_score,
          "\n")

    end_time = time.time()
    time_random = (end_time - start_time) / 60  # 转换为分钟
    print(f"Optimization completed in {time_random} minutes.")

    return best_params, best_score, time_random

# 执行优化
best_params, best_score, time_random = optimized_optuna_search_and_report(100, optuna.samplers.NSGAIISampler())

[I 2023-10-11 22:53:37,577] A new study created in RDB with name: random_GAO


Deleted existing study: random_GAO


Best trial: 0. Best value: 0.95722:   1%|▍                                             | 1/100 [00:15<26:04, 15.81s/it]

[I 2023-10-11 22:53:53,372] Trial 0 finished with value: 0.9572204182222072 and parameters: {'n_estimators': 46, 'max_depth': 43, 'max_features': 9, 'min_samples_leaf': 25, 'min_samples_split': 21}. Best is trial 0 with value: 0.9572204182222072.


Best trial: 0. Best value: 0.95722:   2%|▉                                             | 2/100 [00:21<15:43,  9.62s/it]

...

[I 2023-10-12 00:18:59,959] Trial 97 finished with value: 0.9354080624205847 and parameters: {'n_estimators': 109, 'max_depth': 35, 'max_features': 27, 'min_samples_leaf': 25, 'min_samples_split': 3}. Best is trial 51 with value: 0.9636458711235096.


Best trial: 51. Best value: 0.963646:  99%|████████████████████████████████████████▌| 99/100 [1:26:10<01:05, 65.55s/it]

[I 2023-10-12 00:19:48,061] Trial 98 finished with value: 0.9500340446673186 and parameters: {'n_estimators': 126, 'max_depth': 30, 'max_features': 11, 'min_samples_leaf': 42, 'min_samples_split': 13}. Best is trial 51 with value: 0.9636458711235096.


Best trial: 51. Best value: 0.963646: 100%|████████████████████████████████████████| 100/100 [1:26:37<00:00, 51.97s/it]

[I 2023-10-12 00:20:14,736] Trial 99 finished with value: 0.9527392821847205 and parameters: {'n_estimators': 79, 'max_depth': 37, 'max_features': 10, 'min_samples_leaf': 40, 'min_samples_split': 13}. Best is trial 51 with value: 0.9636458711235096.

 
 best params:  {'n_estimators': 71, 'max_depth': 36, 'max_features': 9, 'min_samples_leaf': 1, 'min_samples_split': 9} 
 
 best score:  [0.9636458711235096] 


 
 best params:  {'n_estimators': 71, 'max_depth': 36, 'max_features': 9, 'min_samples_leaf': 1, 'min_samples_split': 9} 
 
 best score:  [0.9636458711235096] 

Optimization completed in 86.62652257680892 minutes.


# 4. 贝叶斯优化

In [6]:
#（1）构建目标函数及定义参数空间
def optuna_objective(trial):
    
    #定义参数空间
    n_estimators = trial.suggest_int("n_estimators",1,150,1) #整数型，(参数名称，下界，上界，步长)
    max_depth = trial.suggest_int("max_depth",30,50,1)
    max_features = trial.suggest_int("max_features",5,30,1)
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,51,1)
    min_samples_split = trial.suggest_int("min_samples_split",2,22,1)

    #定义评估器
    #需要优化的参数由上述参数空间决定
    #不需要优化的参数则直接填写具体值
    rfc = RFC(n_estimators = n_estimators
              ,max_depth = max_depth
              ,max_features = max_features
              ,min_samples_leaf = min_samples_leaf
              ,min_samples_split = min_samples_split
              ,random_state=0
              ,verbose=False
              ,n_jobs=-1
             )

    #交叉验证过程，输出准确率(accuracy)
    cv = KFold(n_splits=5,shuffle=True,random_state=0)
    score = cross_val_score(rfc,X_dr,y
                            ,scoring='accuracy'
                            ,cv=cv #交叉验证模式
                            ,verbose=False #是否打印进程
                            ,n_jobs=-1 #线程数
                            ,error_score='raise' #如果在某个折叠中出现错误，会引发一个异常
                           )
    #最终输出平均准确率
    return np.mean(score)

#（2）定义优化目标函数的具体流程
def optimizer_optuna(n_trials, algo):

    # 检查是否已经存在study，存在则删除，不存在则略过    
    try:
        storage = "sqlite:///random_BO"  # Specify the storage URL here.
        study_name = "random_BO"
       
        # 获取 study 的 ID
        storage = optuna.storages.get_storage(storage)
        study_id = storage.get_study_id_from_name(study_name)
       
        # 删除已存在的 study
        storage.delete_study(study_id)
        print(f"Deleted existing study: {study_name}")
    except KeyError as e:
        print(f"Study '{study_name}' does not exist.")
    
    #定义使用TPE或者GP
    if algo == "TPE":
        algo = optuna.samplers.TPESampler(n_startup_trials = 15, n_ei_candidates = 12)
    elif algo == "GP":
        from optuna.integration import SkoptSampler
        import skopt
        algo = SkoptSampler(skopt_kwargs={'base_estimator':'GP', #选择高斯过程
                                          'n_initial_points':30, #初始观测点10个
                                          'acq_func':'EI'} #选择的采集函数为EI，期望增量
                           )

    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = algo #要使用的具体算法
                                ,direction="maximize" #优化的方向，可以填写minimize或maximize
                                ,storage="sqlite:///random_BO" # Specify the storage URL here.
                                ,study_name="random_BO"
                               )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(optuna_objective #目标函数
                   ,n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   ,show_progress_bar=True #要不要展示进度条呀？
                  )

    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")

    return study.best_trial.params, study.best_trial.values

#（3）执行优化操作
def optimized_optuna_search_and_report(n_trials, algo):
    start_time = time.time()

    # 进行贝叶斯优化
    best_params, best_score = optimizer_optuna(n_trials, algo)

    # 打印最佳参数和分数
    print("\n","\n","best params: ", best_params,
          "\n","\n","best score: ", best_score,
          "\n")

    end_time = time.time()
    time_bayes = (end_time - start_time) / 60  # 转换为分钟
    print(f"Optimization completed in {time_bayes} minutes.")

    return best_params, best_score, time_bayes

# 执行优化
best_params, best_score, time_bayes = optimized_optuna_search_and_report(100, "TPE")

[I 2023-10-12 09:56:32,600] A new study created in RDB with name: random_BO


Deleted existing study: random_BO


Best trial: 0. Best value: 0.955191:   1%|▍                                            | 1/100 [00:10<18:00, 10.91s/it]

[I 2023-10-12 09:56:43,496] Trial 0 finished with value: 0.9551911415954351 and parameters: {'n_estimators': 37, 'max_depth': 35, 'max_features': 8, 'min_samples_leaf': 31, 'min_samples_split': 18}. Best is trial 0 with value: 0.9551911415954351.


Best trial: 1. Best value: 0.960602:   2%|▉                                            | 2/100 [00:43<38:42, 23.70s/it]

...

[I 2023-10-12 11:01:55,262] Trial 97 finished with value: 0.9648293745610381 and parameters: {'n_estimators': 99, 'max_depth': 45, 'max_features': 9, 'min_samples_leaf': 1, 'min_samples_split': 7}. Best is trial 25 with value: 0.965167426491219.


Best trial: 25. Best value: 0.965167:  99%|████████████████████████████████████████▌| 99/100 [1:06:10<00:44, 44.55s/it]

[I 2023-10-12 11:02:42,990] Trial 98 finished with value: 0.9633920641076277 and parameters: {'n_estimators': 111, 'max_depth': 45, 'max_features': 9, 'min_samples_leaf': 2, 'min_samples_split': 8}. Best is trial 25 with value: 0.965167426491219.


Best trial: 25. Best value: 0.965167: 100%|████████████████████████████████████████| 100/100 [1:06:36<00:00, 39.97s/it]

[I 2023-10-12 11:03:09,089] Trial 99 finished with value: 0.9619548608815156 and parameters: {'n_estimators': 99, 'max_depth': 47, 'max_features': 6, 'min_samples_leaf': 7, 'min_samples_split': 6}. Best is trial 25 with value: 0.965167426491219.

 
 best params:  {'n_estimators': 104, 'max_depth': 43, 'max_features': 8, 'min_samples_leaf': 1, 'min_samples_split': 5} 
 
 best score:  [0.965167426491219] 


 
 best params:  {'n_estimators': 104, 'max_depth': 43, 'max_features': 8, 'min_samples_leaf': 1, 'min_samples_split': 5} 
 
 best score:  [0.965167426491219] 

Optimization completed in 66.61450456380844 minutes.


# 5. 导出数据

In [27]:
df = pd.read_csv('D:/1_科研进展/小论文/小论文图片/优化目标函数变化/random.csv',encoding='utf-8')

In [28]:
df.head()

,0,0.955191142,0.957220418
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,1.0,0.960602,0.922387
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [29]:
x = df.iloc[:,:-1]
x

,0,0.955191142
0,NaN,NaN
1,NaN,NaN
2,1.0,0.960602
3,NaN,NaN
4,NaN,NaN
...,...,...
292,NaN,NaN
293,98.0,0.963392
294,NaN,NaN
295,NaN,NaN
